# Ridge Regression

Selain regresi linear OLS (Ordinary Least Squares), ada teknik regresi linear lainnya yang membantu mengatasi beberapa asumsi dan kelemahan yang terkait dengan regresi linear kuadrat terkecil. Ini termasuk:

* Ridge Regression: Diterapkan di https://github.com/berkmancenter/ridge.
* Lasso Regression: Belum ada implementasi Go yang umum digunakan saat ini. 

Perbedaan mencolok dengan github.com/sajari/regression adalah data variabel independen dan dependen dimasukkan ke dalam github.com/berkmancenter/ridge melalui matriks gonum. Untuk menyertakan intersep dalam model, kita perlu menambahkan kolom eksplisit ke matriks variabel independen input, di mana setiap nilai dalam kolom ini hanya 1.0.

## Training Model

In [19]:
import (
    "github.com/berkmancenter/ridge"
    "github.com/gonum/matrix/mat64"
)

In [21]:
func training(file string) (*ridge.RidgeRegression, error) {
    f, err := os.Open(file) 
    if err != nil { 
        return nil, err 
    }
    
    defer f.Close() 
    
    reader := csv.NewReader(f)
    reader.FieldsPerRecord = -1

    csvData, err := reader.ReadAll() 
    if err != nil { 
        return nil, err
    }
    
    // delete header
    csvData = csvData[1:]

    // featureData will hold all the float values that will eventually be 
    // used to form our matrix of features. 
    featureData := make([]float64, 4*len(csvData)) 
    yData := make([]float64, len(csvData)) 
    
    // featureIndex and yIndex will track the current index of the matrix values. 
    var featureIndex int 
    var yIndex int
    
    for _, record := range csvData {
        for i, val := range record { 
            // Convert the value to a float. 
            valParsed, err := strconv.ParseFloat(val, 64) 
            if err != nil { 
                return nil, err
            }
            
            if i < 3 { 
                // Add an intercept to the model. 
                if i == 0 { 
                    featureData[featureIndex] = 1 
                    featureIndex++ 
                }
                
                // Add the float value to the slice of feature floats. 
                featureData[featureIndex] = valParsed 
                featureIndex++ 
            }
            
            if i == 3 { 
                // Add the float value to the slice of y floats. 
                yData[yIndex] = valParsed 
                yIndex++ 
            } 
        }
    }
    
    features := mat64.NewDense(len(csvData), 4, featureData) 
    y := mat64.NewVector(len(csvData), yData)

    // Create a new RidgeRegression value, where 1.0 is the penalty value. 
    r := ridge.New(features, y, 1.0)

    // Train our regression model. 
    r.Regress()

    return r, nil
}

In [23]:
%%
r, err := training("01-shuffle-training.csv")
if err != nil {
    log.Fatal(err)
}

c1 := r.Coefficients.At(0, 0) 
c2 := r.Coefficients.At(1, 0) 
c3 := r.Coefficients.At(2, 0) 
c4 := r.Coefficients.At(3, 0) 
fmt.Printf("\nRegression formula:\n") 
fmt.Printf("y = %0.3f + %0.3f TV + %0.3f Radio + %0.3f Newspaper\n\n", c1, c2, c3, c4)


Regression formula:
y = 3.146 + 0.044 TV + 0.184 Radio + 0.001 Newspaper



In [24]:
func predict(tv float64, radio float64, newspaper float64) float64 {
    return 3.146 + tv * 0.044  + radio * 0.184 + newspaper * 0.001
}

## Evaluasi

In [25]:
func evaluasi(file string) (float64, error) {
    var mAE float64
    f, err := os.Open(file) 
    if err != nil { 
        return mAE, err
    }
    
    defer f.Close() 
    
    reader := csv.NewReader(f)
    reader.FieldsPerRecord = -1
    testData, err := reader.ReadAll() 
    if err != nil { 
        return mAE, err
    }
    
    // delete header
    testData = testData[1:]
    
    for _, record := range testData {
        yObserved, err := strconv.ParseFloat(record[3], 64) 
        if err != nil { 
            return mAE, err
        }
    
        tvVal, err := strconv.ParseFloat(record[0], 64) 
        if err != nil { 
            return mAE, err
        }

        radioVal, err := strconv.ParseFloat(record[1], 64) 
        if err != nil { 
            return mAE, err
        }

        newspaperVal, err := strconv.ParseFloat(record[2], 64) 
        if err != nil { 
            return mAE, err
        }
        
        // Predict y with our trained model. 
        yPredicted := predict(tvVal, radioVal, newspaperVal)
    
        // Add the to the mean absolute error. 
        mAE += math.Abs(yObserved-yPredicted) / float64(len(testData))
    }

    return mAE, nil
}

In [26]:
%%
mAE, err := evaluasi("./01-shuffle-test.csv")
if err != nil {
    log.Fatal(err)
}

fmt.Printf("MAE = %0.2f\n\n", mAE)

MAE = 1.25



Perlu diperhatikan bahwa menambahkan Iklan di Surat Kabar ke dalam model tidak benar-benar meningkatkan MAE (Mean Absolute Error) kita. Oleh karena itu, menambahkan variabel ini bukanlah keputusan yang baik dalam kasus ini. Hal ini dikarenakan penambahan variabel ini meningkatkan kompleksitas model tanpa memberikan perubahan signifikan pada performa model kita.

Setiap peningkatan kompleksitas atau kecanggihan yang Anda tambahkan ke model harus disertai dengan justifikasi yang terukur. Menggunakan model yang canggih hanya karena menarik secara intelektual dapat menimbulkan masalah di kemudian hari.